- pip install google-api-python-client
- source: https://developers.google.com/youtube/v3/quickstart/python?hl=tr

In [1]:
from googleapiclient.discovery import build
from IPython.display import JSON
import pandas as pd

In [ ]:
from config import YOUTUBE_API_KEY

In [3]:
api_service_name = "youtube"
api_version = "v3"

channel_ids = [
    "UCYO_jab_esuFRV4b17AJtAw",
    # Can add more channels
]

# Get credentials and create an API client

youtube = build(
    api_service_name, api_version, developerKey=YOUTUBE_API_KEY)

request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=",".join(channel_ids)
)
response = request.execute()

print(response)

{'kind': 'youtube#channelListResponse', 'etag': 'cv5bbAnSvVfrZP5oZePS2xSRDrw', 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#channel', 'etag': 'NAsYl4ngGFkpuAYGeQi-UAKuFPs', 'id': 'UCYO_jab_esuFRV4b17AJtAw', 'snippet': {'title': '3Blue1Brown', 'description': 'My name is Grant Sanderson. Videos here cover a variety of topics in math, or adjacent fields like physics and CS, all with an emphasis on visualizing the core ideas. To goal is to use animation to help elucidate and motivate otherwise tricky topics, and for difficult problems to be made simple with changes in perspective.\n\nFor more information, other projects, FAQs, and inquiries see the website: https://www.3blue1brown.com', 'customUrl': '@3blue1brown', 'publishedAt': '2015-03-03T23:11:55Z', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AIf8zZSApZdSBilmxjGARJbX1WdIFYXFv69aixy9FcJraA=s88-c-k-c0x00ffffff-no-rj', 'width': 88, 'height': 88}, 'medium': {'url': 'https://yt3.ggpht.co

In [4]:
JSON(response)

<IPython.core.display.JSON object>

In [5]:
api_service_name = "youtube"
api_version = "v3"

channel_ids = [
    "UCYO_jab_esuFRV4b17AJtAw",
    # Can add more channels
]

# Get credentials and create an API client
youtube = build(api_service_name, api_version, developerKey=YOUTUBE_API_KEY)

In [6]:
def get_channel_stats(youtube, channel_ids):
    all_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=",".join(channel_ids)
    )

    response = request.execute()

    # loop through items in response
    for item in response["items"]:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }
        all_data.append(data)
    
    return pd.DataFrame(all_data)

In [7]:
channel_ids = get_channel_stats(youtube, channel_ids)

In [8]:
channel_ids

,channelName,subscribers,views,totalVideos,playlistId
0,3Blue1Brown,5760000,425321114,161,UUYO_jab_esuFRV4b17AJtAw


In [9]:
request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId="UUYO_jab_esuFRV4b17AJtAw"
    )
response = request.execute()

In [10]:
JSON(response)

<IPython.core.display.JSON object>

In [11]:
def get_video_ids(youtube, playlist_id):
    video_ids = []

    request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id
        )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
        
    return video_ids

In [12]:
playlist_id = "UUYO_jab_esuFRV4b17AJtAw"

video_ids = get_video_ids(youtube, playlist_id)

In [13]:
video_ids

['BTryyW7gqg4', 'kpG7I2MOcnI', '3KbiJ-z0j44', 'R1e7aHBKLnM', 'tjIOqIr80ns']

In [14]:
def get_video_ids(youtube, playlist_id):
    video_ids = []

    request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id,
            maxResults=50
        )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    
    while next_page_token is not None:
        request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                playlistId=playlist_id,
                maxResults=50,
                pageToken=next_page_token
            )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')    

        
    return video_ids

In [15]:
playlist_id = "UUYO_jab_esuFRV4b17AJtAw"

video_ids = get_video_ids(youtube, playlist_id)

In [16]:
len(video_ids)

161